In [ ]:
import logging
# 配置日志
logging.basicConfig(level=logging.INFO)
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Replace 'your_model_file' with the path to the model file you downloaded.
model_id = "mistralai/Mistral-7B-Instruct-v0.2"
tokenizer = AutoTokenizer.from_pretrained(model_id, device_map="mps", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="mps", trust_remote_code=True, torch_dtype=torch.float16)

In [ ]:
# 构造消息
messages = [
    {"role": "user", "content": "你最喜欢的调料是什么？"},
    {"role": "assistant", "content": "嗯，我特别偏爱新鲜的柠檬汁。它能给我烹饪的食物增添恰到好处的鲜味！"},
    {"role": "user", "content": "你有没有美食的食谱？"}
]

# 应用消息模板
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
print(tokenizer.decode(inputs[0]))
inputs = tokenizer(tokenizer.decode(inputs[0]), return_tensors="pt")
inputs.to(model.device)
outputs = model.generate(**inputs, max_new_tokens=4000, do_sample=True, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
# Composing the instruction with the necessary template
instruction = "<s> [INST] {} [/INST]".format("请详细解释一下什么是MOE混合专家大模型的架构和原理，请使用中文回答。")
inputs = tokenizer(instruction, return_tensors="pt")
inputs.to("mps")
outputs = model.generate(**inputs, max_new_tokens=4000, do_sample=True, num_return_sequences=1, temperature=0.9, top_k=50, top_p=0.95, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [1]:
import logging
from langchain import LangChain, LangChainModel, LangChainTokenizer
import torch

# Configure logs
logging.basicConfig(level=logging.INFO)

# Replace 'your_model_file' with the path to the model file you downloaded.
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

# Initialize the tokenizer and model
tokenizer = LangChainTokenizer.from_pretrained(model_id, device_map="mps", trust_remote_code=True)
model = LangChainModel.from_pretrained(model_id, device_map="mps", trust_remote_code=True, torch_dtype=torch.float16)

# Construct the message
messages = [
    {"role": "user", "content": "What's your favorite spice?"},
    {"role": "assistant", "content": "Well, I have a preference for fresh lemonade. It adds just the right amount of umami to the food I cook! "},
    {"role": "user", "content": "Do you have a recipe for a good meal?"}
]

# Apply a message template
inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
print(tokenizer.decode(inputs[0]))
inputs = tokenizer(tokenizer.decode(inputs[0]), return_tensors="pt")
inputs.to(model.device)
outputs = model.generate(**inputs, max_new_tokens=4000, do_sample=True, num_return_sequences=1, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

# Composing the instruction with the necessary template
instruction = " [INST]<s> {} [/INST]".format("Please explain in detail what is the architecture and principle of the MOE hybrid expert large model, please answer in Chinese.")
inputs = tokenizer(instruction, return_tensors="pt")
inputs.to("mps")
outputs = model.generate(**inputs, max_new_tokens=4000, do_sample=True, num_return_sequences=1, temperature=0.9, top_k=50, top_p=0.95, pad_token_id=tokenizer.eos_token_id)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))



ImportError: cannot import name 'LangChain' from 'langchain' (/Users/xujiantong/miniconda3/lib/python3.10/site-packages/langchain/__init__.py)

In [ ]:
HUGGINGFACEHUB_API_TOKEN = "f_zaSzRmIZVmpJTRpKKWwTyYxpQvsdMpnwWQ"
import os

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
question = "Who won the FIFA World Cup in the year 1994? "

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

print(llm_chain.run(question))
